In [10]:
import GPflow
import numpy as np

# Viewing, getting and setting parameters.

In [11]:
#build a very simple GPR model
X = np.random.rand(20,1)
Y = np.sin(12*X) + 0.66*np.cos(25*X) + np.random.randn(20,1)*0.01
m = GPflow.gpr.GPR(X, Y, kern=GPflow.kernels.Matern32(1) + GPflow.kernels.Linear(1))
m

Name,values,prior,constraint
name.kern.linear.variance,[ 1.],None,+ve
name.kern.matern32.lengthscales,[ 1.],None,+ve
name.kern.matern32.variance,[ 1.],None,+ve
name.likelihood.variance,[ 1.],None,+ve


Todas las variables del modelo son inicializadas en 1. Para asignar los parámetros de las variables

In [12]:
m.kern.matern32.lengthscales = 0.5
m.likelihood.variance=0.01
m

Name,values,prior,constraint
name.kern.linear.variance,[ 1.],None,+ve
name.kern.matern32.lengthscales,[ 0.5],None,+ve
name.kern.matern32.variance,[ 1.],None,+ve
name.likelihood.variance,[ 0.01],None,+ve


# Constraints and fixes.

Above, all the variables are constrained positive (see right hand table column), the unconstrained representation is given by α=log(exp(θ)−1)


In [13]:
print(m.get_free_state())

[ 0.54132327 -0.43275467  0.54132327 -4.60026653]


Constraints are handled by the Transform classes. You might prefer the constrain α=log(θ): this is easily done by changing setting the transform attribute on a parameter:

In [16]:
m.kern.matern32.lengthscales.transform = GPflow.transforms.Exp()
print(m.get_free_state())

[ 0.54132327 -0.69314918  0.54132327 -4.60026653]


Another helpful feature is the ability to fix parameters. This is done by simply setting the fixed boolean to True: a ‘fixed’ notice appears in the representation and the corresponding variable is removed from the free state.

In [18]:
m.kern.linear.variance.fixed = True
m

Name,values,prior,constraint
name.kern.linear.variance,[ 1.],None,[FIXED]
name.kern.matern32.lengthscales,[ 0.5],None,+ve
name.kern.matern32.variance,[ 1.],None,+ve
name.likelihood.variance,[ 0.01],None,+ve


In [29]:
print(m.get_free_state())

[ 0.54132327 -0.69314918  0.54132327 -4.60026653]


To unfix a parameter, just flip the boolean back. The transformation (+ve) reappears.

In [31]:
m.kern.linear.variance.fixed = False
m

Name,values,prior,constraint
name.kern.linear.variance,[ 1.],None,+ve
name.kern.matern32.lengthscales,[ 0.5],None,+ve
name.kern.matern32.variance,[ 1.],None,+ve
name.likelihood.variance,[ 0.01],None,+ve


# Priors

Priors are set just like transforms and fixes, using members of the GPflow.priors. module. Let’s set a Gamma prior on the RBF-variance.

In [33]:
m.kern.matern32.variance.prior = GPflow.priors.Gamma(2,3)
m



Name,values,prior,constraint
name.kern.linear.variance,[ 1.],None,+ve
name.kern.matern32.lengthscales,[ 0.5],None,+ve
name.kern.matern32.variance,[ 1.],"Ga([ 2.],[ 3.])",+ve
name.likelihood.variance,[ 0.01],None,+ve


# Optimization

Optimization is done by calling m.optimize() which has optional arguments that are passed through to scipy.optimize.minimize (we minimize the negative log-likelihood). Variables that have priors are MAP-estimated, others are ML, i.e. we add the log prior to the log likelihood.

In [34]:
m.optimize()
m


Name,values,prior,constraint
name.kern.linear.variance,[ 4.53115029e-06],None,+ve
name.kern.matern32.lengthscales,[ 0.16749912],None,+ve
name.kern.matern32.variance,[ 1.63360529],"Ga([ 2.],[ 3.])",+ve
name.likelihood.variance,[ 1.00000000e-06],None,+ve
